In [1]:
import csv
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
header ={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"} 
amazon = 'https://www.amazon.in'
amazon_sk = 'https://www.amazon.in/s?k='

In [3]:
def get_amazon_url(title):
    html_data = requests.get(amazon_sk+title,headers=header).text
    soup = BeautifulSoup(html_data, 'lxml')
    items = soup.find_all('div', {'data-component-type': 's-search-result'})
    link = items[0].h2.find_all('a')
    return amazon+link[0].get('href')

In [4]:
def add_data(title):
    try:
        product_page = get_amazon_url(title)
        ht_data = requests.get(product_page, headers=header).text
        soup = BeautifulSoup(ht_data, 'lxml')
        f1, i1 = get_price(soup)
        f2, i2 = get_ratings(soup)
        f3, i3 = get_reviews(soup)
        f4, i4 = get_description(soup)
        
        if(f1 and f2 and f3 and f4):
            return 1, [i1,i2,i3], i4
    except:
        return 0, None

In [5]:
def get_price(soup):
    try:
        price_box = soup.find_all('div', {'id':'desktop_buybox'})
        price = price_box[0].h5.find_all('span', {'id':'price'})[0].text
        return 1, price
    except:
        return 0, None

In [6]:
def get_ratings(soup):
    try:
        rev = soup.find_all('div', {'id':'averageCustomerReviews'})
        stars = rev[0].find_all('a')
        return 1, str([stars[0].text.strip(), stars[1].text.strip()])
    except:
        return 0, None

In [7]:
def get_description(soup):
    try:
        desc = soup.find_all('div', {'id':'bookDescription_feature_div'})
        description = desc[0].text.strip()
        return 1, description
    except:
        return 0, None

In [8]:
def get_reviews(soup):
    try:
        reviews = soup.find_all('div', {'data-hook':'review'})
        review_list = []
        for review_div in reviews:
            review_list.append(review_div.find_all('div', {'data-hook':'review-collapsed'})[0].text.strip("\n"))
            if(len(review_list)>=3):
                review_list = review_list[:3]
        return 1, str(review_list)
    except:
        return 0, None

In [13]:
count = 0;
with open('books.csv', newline='') as f:
    reader = csv.reader(f, delimiter=';')
    for row in reader:
        if(count>4):
            break
        print(row)
        print(type(row))
        print("\n==================================\n")
        count+=1

['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
<class 'list'>


['0195153448', 'Classical Mythology', 'Mark P. O. Morford', '2002', 'Oxford University Press', 'http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg']
<class 'list'>


['0002005018', 'Clara Callan', 'Richard Bruce Wright', '2001', 'HarperFlamingo Canada', 'http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg']
<class 'list'>


['0060973129', 'Decision in Normandy', "Carlo D'Este", '1991', 'HarperPerennial', 'http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg', 'http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg', 'http://images.amazon.com/images/P/0060973129.01.LZZZ

In [12]:
w = open('books_v2.csv', 'w')
desc = open('description.txt', 'w')
new_books = csv.writer(w, delimiter=';')
column_names = ['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Image-URL-S','Image-URL-M','Image-URL-L','Price','Rating','Reviews']
new_books.writerow(column_names)
count = 0
with open('books.csv', newline='', encoding='latin-1') as f:
    reader = csv.reader(f, delimiter=';')
    for row in tqdm(reader):
        try:
            if(count>30):
                break
            if(count==0):
                count+=1
            else:
                title = row[1]
                state, entry, description = add_data(title)
                if(state):
                    print(title+" :: STATUS : OK")
                    count+=1
                    row.extend(entry)
                    new_books.writerow(row)
                    desc.write(description+'\n')
                else:
                    print(title+" :: STATUS : INFO MISSING")
        except:
            pass
w.close()
desc.close()

2it [00:06,  3.25s/it]

Classical Mythology :: STATUS : OK


3it [00:11,  4.06s/it]

Clara Callan :: STATUS : OK


5it [00:23,  5.13s/it]

...

Left Behind: A Novel of the Earth's Last Days (Left Behind #1) :: STATUS : OK


54it [05:13,  6.48s/it]

The Street Lawyer :: STATUS : OK


55it [05:20,  5.82s/it]

Love, Medicine and Miracles :: STATUS : OK
